# LangChain and Embeddings
This doesn't work yet.

In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# Fix bug that loads too old of version of sqlite3.
# https://docs.trychroma.com/troubleshooting#sqlite
# https://gist.github.com/defulmere/8b9695e415a44271061cc8e272f3c300?permalink_comment_id=4691192#gistcomment-4691192
# Edit .venv/lib/python3.11/site-packages/chromadb/__init__.py


In [18]:
# https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/
raw_documents = TextLoader("./llama-data/data/apache_logs/1.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

embeddings = OllamaEmbeddings(
    base_url="http://g005:11434",
    model="llama3.1",
)

db = Chroma.from_documents(
    documents, embeddings, persist_directory="llama-data/embeddings/chroma"
)

In [20]:
query = "*"
docs = db.search(query=query, search_type="similarity")

data = docs[0].page_content

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [22]:
model = OllamaLLM(model="llama3.1:70b")

template = """
Question: Can you find any anomalies in this data?
Data: {data}
Answer: There are <number of lines> in this data. I have found the following anomalies on:
    <line>: <message>
"""
prompt = ChatPromptTemplate.from_template(template=template)
model = OllamaLLM(model="llama3.1:70b", base_url="http://g005:11434")
chain = prompt | model
chain.invoke({"data": data})

'There are 17 lines in this data. I have found the following anomalies:\n\n    line 1 and line 2: The user agent string contains "Windows NT 6.1; WOW64", which is unusual because Windows NT 6.1 is an old operating system, and WOW64 is a compatibility layer for running 32-bit applications on 64-bit systems.\n    line 3: The request was made by Googlebot/2.1, but the user agent string claims to be an iPhone running iOS 6.0, which is suspicious because Googlebot is a web crawler and not typically associated with mobile devices.\n    lines 4-5: Tiny Tiny RSS/1.11 (http://tt-rss.org/) made multiple requests in quick succession, which could indicate that the feed was being scraped or monitored by an automated tool.\n    line 6: The request was made using HTTP/1.0 instead of a more modern version like HTTP/1.1 or HTTP/2, which is unusual for most web browsers and clients.\n    lines 7-8: Tiny Tiny RSS/1.11 (http://tt-rss.org/) made multiple requests in quick succession again, similar to lines

# LLama Index


In [33]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    get_response_synthesizer,
    Settings
)
from llama_index.core.readers.base import BaseReader
from llama_index.core import Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama


In [37]:
class TxtReader(BaseReader):
    def load_data(self, file, extra_info=None):
        with open(file, "r") as f:
            text = f.read()
        # load_data returns a list of Document objects
        return [Document(text=text, extra_info=extra_info or {})]


reader = SimpleDirectoryReader(
    input_files=["llama-data/data/apache_logs/2.txt"],
    file_extractor={".txt": TxtReader()},
)
documents = reader.load_data()

In [38]:
Settings.llm = Ollama(model="llama3.1:latest", request_timeout=360.0)
Settings.embed_model = OllamaEmbedding(
    model_name="llama3.1:70b",
    base_url="http://g005:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [39]:
# build index
index = VectorStoreIndex.from_documents(documents)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=100
)
response_synth = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synth,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine.query("How many lines are in the loaded data?")
print(response)

Empty Response
